In [53]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_validate
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('code_spark_and_datasets/fraud_detection.csv') 
data = data.reset_index(drop = True)
df = data.drop('Unnamed: 0', axis = 1).copy()
df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [3]:
df['TX_DATETIME'] = pd.to_datetime(df['TX_DATETIME'])

In [4]:
df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0
...,...,...,...,...,...,...,...,...,...
1763638,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0
1763639,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0
1763640,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0
1763641,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0


Ce que chaque ligne résume essentiellement est que, à 00:00:31, le 1er avril 2018, un client avec l'ID 596 a effectué un paiement d'une valeur de 57,19 à un commerçant avec l'ID 3156, et que la transaction n'était pas frauduleuse. Ensuite, à 00:02:10, le 1er avril 2018, un client avec l'ID 4961 a effectué un paiement d'une valeur de 81,51 à un commerçant avec l'ID 3412, et que la transaction n'était pas frauduleuse. Et ainsi de suite. L'ensemble de données simulées est une longue liste de ces transactions (1,8 million au total). La variable transaction_ID est un identifiant unique pour chaque transaction.



# Ingénierie des caractéristiques

L'ingénierie des caractéristiques est le processus de transformation des données brutes en caractéristiques qui représentent mieux le problème sous-jacent pour les modèles prédictifs, ce qui permet d'améliorer la précision du modèle sur des données non vues.

Dans cet ensemble de données, les seules caractéristiques numériques et ordonnées sont le montant de la transaction et l'étiquette de la fraude. La date est un horodatage Panda et n'est donc pas numérique. Les identifiants des transactions, des clients et des terminaux sont numériques mais non ordonnés : il serait absurde de supposer, par exemple, que le terminal portant l'identifiant 3548 est "plus grand" ou "plus gros" que le terminal portant l'identifiant 1983. Ces identifiants représentent plutôt des "entités" distinctes, que l'on appelle des caractéristiques catégorielles.

Il n'existe malheureusement aucune procédure standard pour traiter les caractéristiques non numériques ou catégorielles. Ce sujet est connu dans la littérature sur l'apprentissage automatique sous le nom d'ingénierie des caractéristiques ou de transformation des caractéristiques. En substance, l'objectif de l'ingénierie des caractéristiques est de concevoir de nouvelles caractéristiques qui sont supposées être pertinentes pour un problème de prédiction. La conception de ces caractéristiques dépend généralement du problème et implique une connaissance du domaine.

Dans cette section, nous allons mettre en œuvre trois types de transformation de caractéristiques dont on sait qu'elles sont pertinentes pour la détection de la fraude par carte de paiement.


## Binary encoding or One-Hot encoding

Le premier type de transformation concerne la variable date/heure, et consiste à créer des traits binaires qui caractérisent les périodes potentiellement pertinentes. Nous allons créer deux caractéristiques de ce type. La première caractérisera si une transaction a lieu pendant un jour de semaine ou pendant le week-end. La seconde caractérisera si une transaction a lieu le jour ou la nuit. Ces caractéristiques peuvent être utiles car il a été observé dans des ensembles de données du monde réel que les schémas de fraude diffèrent entre les jours de la semaine et les week-ends, et entre le jour et la nuit.

### Transformations de la date et de l'heure

Nous allons créer deux nouvelles caractéristiques binaires à partir des dates et heures des transactions :

* La première caractérisera si une transaction a lieu pendant un jour de semaine (valeur 0) ou un week-end (1), et sera appelée `TX_DURING_WEEKEND`.
* Le second caractérisera si une transaction a lieu pendant la journée (valeur 0) ou pendant la nuit (1). La nuit est définie comme les heures qui sont comprises entre 0h et 6h du matin. Elle sera appelée `TX_DURING_NIGHT`. 

Pour la fonction `TX_DURING_WEEKEND`, nous définissons une fonction `is_weekend` qui prend en entrée un timestamp Panda, et retourne 1 si la date est pendant un week-end, ou 0 sinon. L'objet timestamp fournit de manière pratique la fonction `weekday` pour aider à calculer cette valeur.

In [5]:
def is_weekend(tx_datetime):
    
    # Transformer la date en jour de la semaine (0 est le lundi, 6 est le dimanche)
    weekday = tx_datetime.weekday()
    # Valeur binaire : 0 si jour de semaine, 1 si week-end.
    is_weekend = weekday>=5
    
    return int(is_weekend)


In [6]:
%time df['TX_DURING_WEEKEND']=df.TX_DATETIME.apply(is_weekend)

CPU times: user 8.44 s, sys: 274 ms, total: 8.72 s
Wall time: 8.78 s


In [7]:
def is_night(tx_datetime):
    
    # Obtenez l'heure de la transaction
    tx_hour = tx_datetime.hour
    # Valeur binaire : 1 si l'heure est inférieure à 6, et 0 sinon.
    is_night = tx_hour<=6
    
    return int(is_night)

In [8]:
%time df['TX_DURING_NIGHT']=df.TX_DATETIME.apply(is_night)

CPU times: user 9.13 s, sys: 385 ms, total: 9.51 s
Wall time: 9.91 s


## RFM (Recency Frequency, Monetary value)

Le deuxième type de transformation concerne l'identifiant du client et consiste à créer des caractéristiques permettant d'identifier les comportements d'achat du client. Nous suivrons le cadre RFM (Recency, Frequency, Monetary value) proposé dans, et garderons la trace du montant moyen des dépenses et du nombre de transactions pour chaque client et pour trois tailles de fenêtres. Cela conduira à la création de six nouvelles fonctionnalités.

### Transformations de l'identité du client

Procédons maintenant aux transformations d'identifiants clients. Nous allons nous inspirer du cadre RFM (Recency, Frequency, Monetary value) et calculer deux de ces caractéristiques sur trois fenêtres de temps. La première caractéristique sera le nombre de transactions qui se produisent dans une fenêtre de temps (fréquence). La seconde sera le montant moyen dépensé dans ces transactions (valeur monétaire). Les fenêtres de temps seront fixées à un, sept et trente jours. Cela permettra de générer six nouvelles fonctionnalités. Notez que ces fenêtres temporelles pourront être optimisées ultérieurement avec les modèles à l'aide d'une procédure de sélection de modèles. 

Implémentons ces transformations en écrivant une fonction `get_customer_spending_behaviour_features`. Cette fonction prend en entrée l'ensemble des transactions d'un client et un ensemble de tailles de fenêtres. Elle renvoie un DataFrame avec les six nouvelles caractéristiques. Notre implémentation s'appuie sur la fonction `rolling` de Panda, qui facilite le calcul des agrégats sur une fenêtre de temps.

In [9]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Classons d'abord les transactions par ordre chronologique
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # La date et l'heure de la transaction sont définies comme index, ce qui permet d'utiliser la fonction de roulement. 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # Pour chaque taille de fenêtre
    for window_size in windows_size_in_days:
        
        # Calculer la somme des montants des transactions et le nombre de transactions pour la taille de fenêtre donnée.
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Calculer le montant moyen des transactions pour la taille de la fenêtre donnée.
        # NB_TX_WINDOW est toujours >0 car la transaction en cours est toujours incluse.
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Sauvegarder les valeurs des caractéristiques
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Réindexation selon les ID des transactions
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # Et retourner le dataframe avec les nouvelles caractéristiques
    return customer_transactions


In [10]:
%time df=df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
df=df.sort_values('TX_DATETIME').reset_index(drop=True)

CPU times: user 1min 22s, sys: 1.16 s, total: 1min 23s
Wall time: 1min 25s


## Frequency encoding or risk encoding

Le troisième type de transformation concerne l'identifiant du terminal et consiste à créer de nouveaux traits qui caractérisent le "risque" associé au terminal. Le risque sera défini comme le nombre moyen de fraudes qui ont été observées sur le terminal pour trois tailles de fenêtres. Cela conduira à la création de trois nouvelles caractéristiques.

### Transformations de l'ID terminal

Enfin, procédons aux transformations de l'ID du terminal. L'objectif principal sera d'extraire un *score de risque*, qui évalue l'exposition d'un terminal ID donné aux transactions frauduleuses. Le score de risque sera défini comme le nombre moyen de transactions frauduleuses qui ont eu lieu sur un terminal ID au cours d'une fenêtre de temps. Comme pour les transformations d'identifiants clients, nous utiliserons trois tailles de fenêtres, de 1, 7 et 30 jours.

Contrairement aux transformations de l'identification du client, les fenêtres de temps ne précéderont pas directement une transaction donnée. Au lieu de cela, elles seront décalées d'une *période de retard*. Cette période de retard tient compte du fait que, dans la pratique, les transactions frauduleuses ne sont découvertes qu'après une enquête sur la fraude ou une plainte du client. Par conséquent, les étiquettes de fraude, qui sont nécessaires pour calculer le score de risque, ne sont disponibles qu'après cette période de retard. En première approximation, ce délai sera fixé à une semaine. 

Effectuons le calcul des scores de risque en définissant une fonction `get_count_risk_rolling_window`. La fonction prend comme entrées le DataFrame des transactions pour un ID de terminal donné, la période de retard, et une liste de tailles de fenêtres. Dans la première étape, le nombre de transactions et de transactions frauduleuses est calculé pour la période de retard (`NB_TX_DELAY` et `NB_FRAUD_DELAY`). Dans la deuxième étape, le nombre de transactions et de transactions frauduleuses est calculé pour chaque taille de fenêtre plus la période de retard (`NB_TX_DELAY_WINDOW` et `NB_FRAUD_DELAY_WINDOW`). Le nombre de transactions et de transactions frauduleuses qui se sont produites pour une taille de fenêtre donnée, décalée en arrière par la période de retard, est ensuite obtenu en calculant simplement les différences des quantités obtenues pour la période de retard, et la taille de fenêtre plus la période de retard :

```
NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
```

Le score de risque est finalement obtenu en calculant la proportion de transactions frauduleuses pour chaque taille de fenêtre (ou 0 si aucune transaction n'a eu lieu pour la fenêtre donnée) :

```
RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
```

En plus du score de risque, la fonction renvoie également le nombre de transactions pour chaque taille de fenêtre. Cela se traduit par l'ajout de six nouvelles fonctions : Le risque et le nombre de transactions, pour trois tailles de fenêtres.


In [11]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Remplacer les valeurs NA par 0 (tous les scores de risque non définis où NB_TX_WINDOW est 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions


In [12]:
%time df=df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
df=df.sort_values('TX_DATETIME').reset_index(drop=True)

CPU times: user 3min 13s, sys: 601 ms, total: 3min 14s
Wall time: 3min 16s


In [13]:
df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,...,2.0,57.160000,2.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.00000
1,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.00000
2,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.00000
3,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,...,2.0,81.510000,2.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.00000
4,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763638,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,1,...,12.0,67.047500,72.0,69.521111,1.0,0.0,4.0,0.0,28.0,0.00000
1763639,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,1,...,21.0,22.173810,93.0,24.780753,1.0,0.0,9.0,0.0,39.0,0.00000
1763640,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,1,...,21.0,7.400476,65.0,7.864462,1.0,0.0,5.0,0.0,33.0,0.00000
1763641,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,1,...,16.0,107.052500,51.0,102.919608,1.0,0.0,6.0,0.0,28.0,0.00000


Pourcentage des transaction froduleuse de la base de données

In [26]:
(df['TX_FRAUD'].value_counts()/df.shape[0])*100

0    99.167405
1     0.832595
Name: TX_FRAUD, dtype: float64

In [15]:
df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD',
       'TX_FRAUD_SCENARIO', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT',
       'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW',
       'TERMINAL_ID_NB_TX_1DAY_WINDOW', 'TERMINAL_ID_RISK_1DAY_WINDOW',
       'TERMINAL_ID_NB_TX_7DAY_WINDOW', 'TERMINAL_ID_RISK_7DAY_WINDOW',
       'TERMINAL_ID_NB_TX_30DAY_WINDOW', 'TERMINAL_ID_RISK_30DAY_WINDOW'],
      dtype='object')

In [50]:
X = df.drop(['TRANSACTION_ID', 'TX_DATETIME', 
 'CUSTOMER_ID', 'TERMINAL_ID', 
 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 
 'TX_TIME_DAYS', 'TX_FRAUD', 'TX_FRAUD_SCENARIO'], axis =1)

y = df['TX_FRAUD']

# Creation de modèle


fonction

In [32]:
def kfold_cv_with_classifier(classifier,
                             X,
                             y,
                             n_splits=5,
                             strategy_name="Basline classifier"):
    
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    cv_results_ = cross_validate(classifier, X, y, cv=cv,
                                                    scoring=['roc_auc',
                                                            'average_precision',
                                                            'balanced_accuracy'],
                                                    return_estimator=True)
    results = round(pd.DataFrame(cv_results_),3)
    results_mean = list(results.mean().values)
    results_std = list(results.std().values)
    results_df = pd.DataFrame([[str(round(results_mean[i],3))+'+/-'+
                                str(round(results_std[i],3)) for i in range(len(results))]],
                              columns=['Fit time (s)','Score time (s)',
                                       'AUC ROC','Average Precision','Balanced accuracy'])
    results_df.rename(index={0:strategy_name}, inplace=True)


    return results_df

In [33]:
classifier = DecisionTreeClassifier(max_depth=5, random_state=0)

results_df_dt_baseline = kfold_cv_with_classifier(classifier, X, y, n_splits=5, 
                                                         strategy_name="Decision tree - Baseline")

/tmp/ipykernel_7046/355113265.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_mean = list(results.mean().values)
/tmp/ipykernel_7046/355113265.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_std = list(results.std().values)


In [34]:
results_df_dt_baseline

,Fit time (s),Score time (s),AUC ROC,Average Precision,Balanced accuracy
Decision tree - Baseline,15.454+/-1.765,0.479+/-0.09,0.828+/-0.003,0.639+/-0.008,0.79+/-0.004


## Cost-sensitive learning

L'apprentissage sensible aux coûts est un sous-domaine de l'apprentissage automatique qui traite des problèmes de classification où les coûts de mauvaise classification ne sont pas égaux. Ils sont souvent liés au problème du déséquilibre des classes puisque dans la plupart de ces problèmes, l'objectif est de détecter des événements qui sont rares. Les ensembles de données d'apprentissage contiennent donc généralement moins d'exemples de l'événement d'intérêt.

### la matrice des coûts

Dans la section précédente, nous avons indiqué que la matrice des coûts était la méthode standard pour quantifier les coûts des erreurs de classification. En désignant par ***C*** la matrice des coûts, ses entrées ***C(i,j)*** quantifient le coût de la prédiction de la classe lorsque la vraie classe est . Pour un problème de classification binaire, la matrice des coûts est une matrice ***(2, 2)***.

Les classifications correctes ont un coût de zéro, c'est-à-dire ***C00 = C11 = 0***. Les coûts des classifications erronées sont toutefois difficiles à estimer dans la pratique. Le fait de rater une transaction frauduleuse (faux négatif) entraîne une perte directement liée au montant de la transaction, mais aussi à d'autres utilisations frauduleuses de la carte et à la réputation de l'entreprise. Dans le même temps, le blocage de transactions légitimes (faux positifs) entraîne des désagréments pour les clients, génère des coûts d'investigation inutiles et nuit également à la réputation de l'entreprise.

Dans les problèmes de déséquilibre sensibles aux coûts, l'approche heuristique la plus populaire pour estimer les coûts consiste à utiliser le ratio de déséquilibre (IR). Désignons par ***X*** l'ensemble de données déséquilibré, ***X0*** et ***X1*** étant les sous-ensembles d'échantillons appartenant respectivement à la classe majoritaire et à la classe minoritaire. L'IR de l'ensemble de données ***X*** est défini comme suit :

    -IR=|X1|/|X0|
    
où **|.|** désigne la cardinalité d'un ensemble.

L'utilisation de l'IR pour définir les coûts de mauvaise classification est généralement une bonne heuristique. Elle présente cependant certaines limites, notamment en ce qui concerne les échantillons de petite taille, le chevauchement des classes et les instances bruyantes ou limites. Une pratique complémentaire courante consiste à considérer les coûts de mauvaise classification comme un hyperparamètre à identifier par la sélection de modèles.

Python sklearn fournit un support pour l'apprentissage sensible aux coûts pour la plupart des classificateurs de base grâce au paramètre **class_weight**. Ce paramètre permet de spécifier les coûts de trois manières différentes :

- Aucun : Les coûts des mauvaises classifications sont fixés à 1 (par défaut)

- équilibré : Les coûts sont fixés en fonction du ratio de déséquilibre.

- {0:c10, 1:c01} : Les coûts des erreurs de classification sont explicitement définis pour les deux classes au moyen d'un dictionnaire.

L'utilisation de poids de classe implique généralement une modification de la fonction de perte de l'algorithme d'apprentissage. La modification dépend du type d'algorithme. En pénalisant fortement les erreurs sur la classe minoritaire, l'apprentissage sensible aux coûts améliore leur importance pendant l'étape d'apprentissage du classificateur. Cela éloigne la frontière de décision de ces instances, ce qui permet d'améliorer la généralisation sur la classe minoritaire.


In [36]:
IR= (df['TX_FRAUD'].value_counts()/df.shape[0])[1]/(df['TX_FRAUD'].value_counts()/df.shape[0])[0]
class_weight={0:IR,1:1}


In [37]:
classifier = DecisionTreeClassifier(max_depth=5,class_weight=class_weight,random_state=0)

In [38]:
results_df_dt_cost_sensitive =  kfold_cv_with_classifier(classifier, X, y, n_splits=5, 
                                                         strategy_name="Decision tree - Cost-sensitive")

/tmp/ipykernel_7046/355113265.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_mean = list(results.mean().values)
/tmp/ipykernel_7046/355113265.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_std = list(results.std().values)


In [39]:
pd.concat([results_df_dt_baseline, 
           results_df_dt_cost_sensitive])

,Fit time (s),Score time (s),AUC ROC,Average Precision,Balanced accuracy
Decision tree - Baseline,15.454+/-1.765,0.479+/-0.09,0.828+/-0.003,0.639+/-0.008,0.79+/-0.004
Decision tree - Cost-sensitive,19.026+/-3.016,0.605+/-0.062,0.869+/-0.007,0.636+/-0.011,0.853+/-0.01


### LogisticRegression

In [42]:
classifier = LogisticRegression(C=1,class_weight={0:IR,1:1},random_state=0)

results_df_dt_cost_sensitive_LR =  kfold_cv_with_classifier(classifier, X, y, n_splits=5, 
                                                         strategy_name="Logistic regression - Cost-sensitive")

pd.concat([results_df_dt_baseline, 
           results_df_dt_cost_sensitive,
           results_df_dt_cost_sensitive_LR])

/home/omar/Python-3.8.0/jupyter_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/omar/Python-3.8.0/jupyter_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

,Fit time (s),Score time (s),AUC ROC,Average Precision,Balanced accuracy
Decision tree - Baseline,15.454+/-1.765,0.479+/-0.09,0.828+/-0.003,0.639+/-0.008,0.79+/-0.004
Decision tree - Cost-sensitive,19.026+/-3.016,0.605+/-0.062,0.869+/-0.007,0.636+/-0.011,0.853+/-0.01
Logistic regression - Cost-sensitive,37.46+/-2.721,0.787+/-0.121,0.894+/-0.003,0.505+/-0.005,0.864+/-0.002


## Resampling strategies

In [44]:

# random_state is set to 0 for reproducibility
ROS = RandomOverSampler(sampling_strategy=1, random_state=0)

In [51]:
X_resampled, Y_resampled = ROS.fit_resample(X, y)


In [52]:
Y_resampled.value_counts()

0    1748959
1    1748959
Name: TX_FRAUD, dtype: int64

In [54]:
def kfold_cv_with_sampler_and_classifier(classifier,
                                         sampler_list,
                                         X,
                                         y,
                                         n_splits=5,
                                         strategy_name="Baseline classifier"):
    
    # Create a pipeline with the list of samplers, and the estimator
    estimators = sampler_list.copy()
    estimators.extend([('clf', classifier)])
    
    pipe = Pipeline(estimators)
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    
    cv_results_ = cross_validate(pipe,X,y,cv=cv,
                                                         scoring=['roc_auc',
                                                                  'average_precision',
                                                                  'balanced_accuracy'],
                                                         return_estimator=True)
    
    results = round(pd.DataFrame(cv_results_),3)
    results_mean = list(results.mean().values)
    results_std = list(results.std().values)
    results_df = pd.DataFrame([[str(round(results_mean[i],3))+'+/-'+
                                str(round(results_std[i],3)) for i in range(len(results))]],
                              columns=['Fit time (s)','Score time (s)',
                                       'AUC ROC','Average Precision','Balanced accuracy'])
    results_df.rename(index={0:strategy_name}, inplace=True)
    
    return results_df

In [55]:
sampler_list = [('sampler',RandomOverSampler(sampling_strategy=1, random_state=0))]
classifier = DecisionTreeClassifier(max_depth=5, random_state=0)

results_df_ROS = kfold_cv_with_sampler_and_classifier(classifier, 
                                                    sampler_list, 
                                                    X, y, 
                                                    n_splits=5,
                                                    strategy_name="Decision tree - ROS")

/tmp/ipykernel_7046/1039580869.py:23: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_mean = list(results.mean().values)
/tmp/ipykernel_7046/1039580869.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_std = list(results.std().values)


In [56]:
pd.concat([results_df_dt_baseline, 
           results_df_dt_cost_sensitive,
           results_df_dt_cost_sensitive_LR,
           results_df_ROS])

,Fit time (s),Score time (s),AUC ROC,Average Precision,Balanced accuracy
Decision tree - Baseline,15.454+/-1.765,0.479+/-0.09,0.828+/-0.003,0.639+/-0.008,0.79+/-0.004
Decision tree - Cost-sensitive,19.026+/-3.016,0.605+/-0.062,0.869+/-0.007,0.636+/-0.011,0.853+/-0.01
Logistic regression - Cost-sensitive,37.46+/-2.721,0.787+/-0.121,0.894+/-0.003,0.505+/-0.005,0.864+/-0.002
Decision tree - ROS,26.296+/-1.863,0.482+/-0.018,0.87+/-0.008,0.638+/-0.008,0.853+/-0.009
